# Display and GUI

We will use [littlevgl](https://littlevgl.com/) - a very powerful graphics library for embedded devices. It's worth checking out their website and documentation.

Here we will write a very simple GUI that will turn on and off our LEDs. Just 4 buttons and that's it.

First we need to initialize the display. It will automatically turn it on, initialize littlevgl and draw the first empty screen.

`display` module has two important functions: `init()` and `update(dt)`.
We should call `init()` function once in the very beginning of the program.

`update(dt)` function should be called whenever we need to draw something new or to handle events from the display.
A parameter we pass to `update` is the time in milliseconds from the last update. Calling `update(30)` will make the library think that 30 ms have passed and it will update the screen.

In [1]:
# %serialconnect
%serialconnect --port=/dev/tty.usbmodem3660384B30362 --baud=115200

Connecting to --port=/dev/tty.usbmodem3660384B30362 --baud=115200 
Ready.


In [2]:
import display
import lvgl as lv

display.init()

Now we have a screen we can draw on.

Let's start with a single button to toggle LED1.

In [3]:
# get current active screen
scr = lv.scr_act()
# clean the screen just in case
scr.clean()
# draw a button on this screen
btn = lv.btn(scr)
# add a text label to the button
lbl = lv.label(btn)
# set text to it
lbl.set_text("Toggle LED")
# set text mode that will make the width of the label
# such that text fits in it
lbl.set_long_mode(lv.label.LONG.EXPAND)
# set the fit mode of the button to expand width
# to the label width
btn.set_fit2(lv.FIT.TIGHT, lv.FIT.NONE)
# align the button to the center of the screen
btn.align(None, lv.ALIGN.CENTER, 0, 0)
# update the display to draw the button
display.update(30)

In [4]:
# now we can update the label text
# and see how it changes on the screen
lbl.set_text("LED1")
display.update(30)

Great! We can draw buttons on the screen!

But the screen is not responsive, we need to call `display.update(dt)` every time, and it is annoying, especially when working from jupyter.

Let's use a trick we discussed in the previous section and schedule display updates.

In [5]:
import micropython
def schedule(t):
    """Try to schedule an LED update"""
    try:
        micropython.schedule(display.update, 30)
    except:
        pass

timer = pyb.Timer(4) # timer 4
timer.init(freq=30)  # 30Hz update rate
timer.callback(schedule)

Now if you try to click on the button you will see that it is changing! Great!

Let's write a callback that will toggle the LED. Otherwise why did we make this button?

Callbacks will receive two arguments: an object that triggered an event and event type. We will toggle the LED only when `lv.EVENT.RELEASED` is detected.

In [6]:
import pyb
led = pyb.LED(1)

def toggle_led(obj, event):
    if event == lv.EVENT.RELEASED:
        led.toggle()

btn.set_event_cb(toggle_led)

Wonderful! We control an LED now!

Let's move on and create 4 buttons one per LED. But as Python is object-oriented we can write a class that includes all the functionality we need from a button.
We will also set every button in a toggle mode

In [7]:
class LedButton(lv.btn):
    """A toggle button that controls a LED"""
    def __init__(self, led, text, *args, **kwargs):
        """Pass pyb.LED() and button text as first two parameters,
           then everything else required by normal button
           - a parent and (optionally) an object to copy from.
        """
        super().__init__(*args, **kwargs)
        # add label as a child
        self.lbl = lv.label(self)
        # store text as we will change the text on toggle
        self.text = text
        # store the LED we control
        self.led = led
        # LED is off by default
        self.lbl.set_text("%s: OFF" % text)
        self.led.off()
        # set callback
        self.set_event_cb(self.toggle)
        # make sure the label will fit
        self.lbl.set_long_mode(lv.label.LONG.EXPAND)
        self.set_fit2(lv.FIT.TIGHT, lv.FIT.NONE)
        # set toggle mode
        self.set_toggle(True)

    def toggle(self, obj, event):
        """Callback on every event"""
        if event == lv.EVENT.VALUE_CHANGED:
            # check the state and toggle the LED & button text
            if self.get_state():
                self.lbl.set_text("%s: ON" % self.text)
                self.led.on()
            else:
                self.lbl.set_text("%s: OFF" % self.text)
                self.led.off()

In [8]:
# clean the screen
scr.clean()
# create buttons for all 4 LEDs
for i in range(1,5):
    btn = LedButton(pyb.LED(i), "LED%d" % i, scr)
    # align vertically
    btn.align(None, lv.ALIGN.CENTER, 0, -250+100*i)